### 지니 Top200 차트

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

##### 1. 데이터 가져오기

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
result = requests.get(url)
result.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [3]:
# Chrome User-Agent
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')

##### 2. 원하는 데이터 찾기

In [4]:
trs = soup.select('tr.list')
len(trs)

50

In [5]:
# 개발자 도구에서 찾아주는 Select
# #body-content > div.newest-list > div > table > tbody > tr:nth-child(1)
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

##### 3. 하나를 선택해서 원하는 정보 추출

In [6]:
tr = trs[0]

In [7]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [8]:
rank = tr.select_one('.number').get_text().split('\n')[0].strip()

In [9]:
# img url
img = 'https:' + tr.select_one('.cover > img')['src']

In [10]:
# title
tr.select_one('.info > .title.ellipsis').get_text().strip()

'I AM'

In [11]:
# 19금 노래 제목
tr.select_one('.info > .title.ellipsis').get_text().split('\n')[-1].strip()

'I AM'

In [12]:
# 위의 두 코드를 합치면
# 방법 1)
title = tr.select_one('.info > .title.ellipsis').get_text().split('\n')[-1].strip()
# 방법 2)
try:
    title = tr.select_one('.info > .title.ellipsis').get_text().strip()
except:
    title = tr.select_one('.info > .title.ellipsis').get_text().split('\n')[-1].strip()

In [13]:
# artist
artist = tr.select_one('.info > .artist.ellipsis').get_text().strip()

In [14]:
# album
album = tr.select_one('.info > .albumtitle.ellipsis').get_text()

##### 4. 한 페이지내에 있는 정보

In [15]:
# 방법 1. 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()
    img = 'https:' + tr.select_one('.cover > img')['src']
    title = tr.select_one('.info > .title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.info > .artist.ellipsis').get_text().strip()
    album = tr.select_one('.info > .albumtitle.ellipsis').get_text()
    line.append({'rank': rank, 'img': img, 'title': title, 'artist': artist, 'album': album})

In [22]:
genie1 =  pd.DataFrame(line)
genie1.head()

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Ditto,NewJeans,NewJeans 'OMG'
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans'


In [20]:
# 방법 2. 개별 리스트로 가져오기
rank_list, img_list, title_list, artist_list, album_list = [], [], [], [], []
for tr in trs:
    rank_list.append(tr.select_one('.number').get_text().split('\n')[0].strip())
    img_list.append('https:' + tr.select_one('.cover > img')['src'])
    title_list.append(tr.select_one('.info > .title.ellipsis').get_text().split('\n')[-1].strip())
    artist_list.append(tr.select_one('.info > .artist.ellipsis').get_text().strip())
    album_list.append(tr.select_one('.info > .albumtitle.ellipsis').get_text())

In [24]:
genie2 = pd.DataFrame({
    'rank': rank_list, 'img': img_list, 'title': title_list,
    'artist': artist_list, 'album': album_list
})
genie2.head()

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Ditto,NewJeans,NewJeans 'OMG'
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans'


##### 5. 에러 처리

##### 6. 모든 페이지에 대해서 처리하기

In [25]:
# https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230424&hh=15&rtm=Y&pg=1
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2023, 4, 24, 15, 4, 12, 268013)

In [29]:
ymd = now.strftime('%Y%m%d')        # ymd = f'{now.year}{now.month:02d}{now.day:02d}'
hh = now.strftime('%H')             # hh = f'{now.hour:02d}'
ymd, hh

('20230424', '15')

In [33]:
from tqdm import tqdm
lines = []
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D&rtm=Y'
for pg in tqdm(range(1, 5)):
    url = f'{base_url}&ymd={ymd}&hh={hh}&pg={pg}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()
        img = 'https:' + tr.select_one('.cover > img')['src']
        title = tr.select_one('.info > .title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.info > .artist.ellipsis').get_text().strip()
        album = tr.select_one('.info > .albumtitle.ellipsis').get_text()
        lines.append({'rank': rank, 'img': img, 'title': title, 'artist': artist, 'album': album})

100%|██████████| 4/4 [00:03<00:00,  1.27it/s]


In [36]:
genie = pd.DataFrame(lines)
genie.head(200).tail(50)

,rank,img,title,artist,album
150,151,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,해금,Agust D,D-DAY
151,152,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,KNOCK,이채연,Over The Moon
152,153,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,마지막 사랑,신예영,마지막 사랑
153,154,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,12 : 45 (Stripped),etham,12 : 45 (Stripped)
154,155,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,빛이 나는 사람,김호중,빛이 나는 사람
155,156,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,너에게 못했던 내 마지막 말은,다비치,너에게 못했던 내 마지막 말은
156,157,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,그댄 행복에 살텐데 (2022),최유리,그댄 행복에 살텐데 (2022)
157,158,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Zero,NewJeans,Zero
158,159,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,내 손을 잡아,아이유 (IU),최고의 사랑 OST Part.4 (MBC 수목드라마)
159,160,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,아로하,조정석,슬기로운 의사생활 OST Part 3 (tvN 목요드라마)


In [37]:
genie.to_csv(f'data/{ymd}{hh}_지니차트.csv', index=False)

In [39]:
pd.read_csv(f'data/{ymd}{hh}_지니차트.csv')

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Ditto,NewJeans,NewJeans 'OMG'
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,CHRISTIAN,Zior Park,WHERE DOES SASQUATCH LIVE? PART 1
...,...,...,...,...,...
195,196,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Dive,NCT 도재정,Perfume - The 1st Mini Album
196,197,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,너를 생각해,주시크 (Joosiq),너를 생각해
197,198,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,언제나 사랑해,케이시 (Kassy),조영수 리메이크 프로젝트 Part.2
198,199,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,시작,가호 (Gaho),이태원 클라쓰 OST Part.2 (JTBC 금토드라마)
